Question 1 Part (a)

In [56]:
import numpy as np
import scipy.stats as sts
import statistics 
import pandas as pd

# Load in and display the data
#macroseries = np.loadtxt('MacroSeries.txt', dtype={'names': ('c_t', 'k_t', 'w_t', 'r_t'), 'formats': ('|S15', '|S15', '|S15', '|S15')}, delimiter = ',')
macroseries = pd.read_csv('Macroseries.csv', header=0)
#print(macroseries)
#macroseries.shift(periods=1)
#print(macroseries['k_t'])


beta = 0.99

def eq_3(w_t, k_t, alpha):

    z_t = np.log(w_t/((1-alpha)*k_t**alpha))

    return z_t

def norm_pdf_z(z_t, rho, sigma, mu):

    z_t_1 = mu + z_t[0:98]
    pdf_vals    = (1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z_t - rho*z_t_1 - (1-rho)*mu)**2 / (2 * sigma**2)))
    
    return pdf_vals

def log_lik_norm(z_t, rho, sigma, mu):
    
    pdf_vals = norm_pdf_z(z_t, rho, sigma, mu)
    ln_pdf_vals = np.log(pdf_vals)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, *args):
    alpha, rho, sigma, mu = params
    w_t, k_t = args
    z_t = eq_3(w_t, k_t, alpha)
    log_lik_val = log_lik_norm(z_t, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

# Set up initial guesses
alpha_0 = 0.5
rho_0 = 0.5
sigma_0 = 1
mu_0 = 1

print('Initial alpha_0=', alpha_0, 'Initial rho_0=', rho_0, 'Initial sigma_0=', sigma_0, 'Initial mu_0=', mu_0)

import scipy.optimize as opt

mle_args = (macroseries['w_t'], macroseries['cap_t'])
params_init = np.array([alpha_0,rho_0,sigma_0,mu_0])
results = opt.minimize(crit, params_init, args=(mle_args), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B', options ={'ftol': 1e-10})
print(results)

alpha_MLE, rho_MLE, sigma_MLE, mu_MLE = results.x
inv_hess = results.hess_inv*np.matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
max_log_lik = log_lik_norm(eq_3(macroseries['w_t'], macroseries['cap_t'], alpha_MLE),rho_MLE,sigma_MLE,mu_MLE)
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, 'sigma_MLE=', sigma_MLE, 'mu_MLE', mu_MLE, 'log Likelihood=', max_log_lik)
print('Inverse Hessian=', inv_hess)

Initial alpha_0= 0.5 Initial rho_0= 0.5 Initial sigma_0= 1 Initial mu_0= 1
      fun: 85.64331517837917
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  84.77693143,  463.58822487,   60.8031371 ,   99.98203438])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 45
      nit: 5
   status: 0
  success: True
        x: array([ 0.84608684,  0.9088313 ,  0.72637296,  0.96116244])
alpha_MLE= 0.846086841185  rho_MLE= 0.90883130443 sigma_MLE= 0.726372963996 mu_MLE 0.961162436921 log Likelihood= -85.64331517837917
Inverse Hessian= [[  3.05790874  -0.13766192  -7.71041112  -2.84578115]
 [ -0.13766192   0.0699859    0.21411023  -0.1720417 ]
 [ -7.71041112   0.21411023  19.77958455   7.81248462]
 [ -2.84578115  -0.1720417    7.81248462   4.06514147]]


Question 1 Part (b)

In [57]:
def eq_4(r_t, k_t, alpha):

    z_t = np.log(r_t/(alpha*k_t**(alpha-1)))

    return z_t

def crit_b(params, *args):
    alpha, rho, sigma, mu = params
    r_t, k_t = args
    z_t = eq_4(r_t, k_t, alpha)
    log_lik_val = log_lik_norm(z_t, rho, sigma, mu)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

# Set up initial guesses
alpha_0_b = 0.5
rho_0_b = 0.5
sigma_0_b = 1
mu_0_b = 1

print('Initial alpha_0=', alpha_0_b, 'Initial rho_0=', rho_0_b, 'Initial sigma_0=', sigma_0_b, 'Initial mu_0=', mu_0_b)

mle_args_b = (macroseries['r_t'], macroseries['cap_t'])
params_init_b = np.array([alpha_0_b,rho_0_b,sigma_0_b,mu_0_b])
results_b = opt.minimize(crit_b, params_init_b, args=(mle_args_b), bounds = ((1e-10, 1-1e-10), (-1+1e-10, 1-1e-10), (1e-10, None), (1e-10, None)), method = 'L-BFGS-B', options ={'ftol': 1e-10})
print(results_b)

alpha_MLE_b, rho_MLE_b, sigma_MLE_b, mu_MLE_b = results_b.x
inv_hess_b = results_b.hess_inv*np.matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
max_log_lik_b = log_lik_norm(eq_4(macroseries['r_t'], macroseries['cap_t'], alpha_MLE_b),rho_MLE_b,sigma_MLE_b,mu_MLE_b)
print('alpha_MLE=', alpha_MLE_b, ' rho_MLE=', rho_MLE_b, 'sigma_MLE=', sigma_MLE_b, 'mu_MLE', mu_MLE_b, 'log Likelihood=', max_log_lik_b)
print('Inverse Hessian=', inv_hess_b)

Initial alpha_0= 0.5 Initial rho_0= 0.5 Initial sigma_0= 1 Initial mu_0= 1
      fun: 14.479294972552593
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  -7.80970026,   -8.52887467,  211.66600686,   -2.1006171 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 65
      nit: 7
   status: 0
  success: True
        x: array([ 0.78889256,  0.78464605,  0.4619325 ,  0.76746017])
alpha_MLE= 0.78889255631  rho_MLE= 0.784646049212 sigma_MLE= 0.461932501198 mu_MLE 0.767460172386 log Likelihood= -14.479294972552593
Inverse Hessian= [[ 0.51725443 -0.48065109  0.17122068  0.04072481]
 [-0.48065109  0.52309855 -0.01153692 -0.06440323]
 [ 0.17122068 -0.01153692  2.6110533   2.58026077]
 [ 0.04072481 -0.06440323  2.58026077  3.03446242]]


Question 1 Part (c)

In [64]:
z_star = eq_4(1,7500000,alpha_MLE)
print('z*=', z_star)

def norm_pdf_z_c(z_t, rho, sigma, mu):

    pdf_vals    = (1/(sigma * np.sqrt(2 * np.pi)) *
                    np.exp( - (z_t - rho*10 - (1-rho)*mu)**2 / (2 * sigma**2)))
    
    return pdf_vals

import scipy.integrate as integrate

prob_r_1 = 1 - integrate.quad(lambda x: norm_pdf_z_c(x, rho_MLE, sigma_MLE, mu_MLE), 0, z_star)[0]
print('Probability that the interest rate this period will be greater than 1:', prob_r_1)

z*= 2.60364223471
Probability that the interest rate this period will be greater than 1: 1.0
